In [40]:
from omegaconf import OmegaConf as om
from omegaconf import DictConfig
from typing import cast
import sys 
import os
sys.path.append(os.path.abspath(".."))

from main import build_model
import src.create_bert as bert_module
import src.create_model as model_module


yaml_path = "../yamls/pretrain/micro_dna_monarch-mixer-pretrain-786dim-80m-parameters.yaml"

with open(yaml_path) as f:
    cfg = om.load(f)
cfg = cast(DictConfig, cfg)
print(cfg.max_duration)
model = model_module.create_model(cfg.model.get("model_config"))
model = bert_module.create_bert_mlm(model_config = cfg.model.get("model_config", None))

loading configuration file config.json from cache at /home/lukas/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertConfig {
  "alibi_starting_size": 512,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bidirectional": true,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "hyena_filter_dropout": 0.2,
  "hyena_filter_order": 128,
  "hyena_training_additions": false,
  "hyena_w_mod": 1,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "monarch_mixer_sequence_mixing": true,
  "monarch_mlp_nblocks": 4,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pool_all": false,
  "position_embedding_type": "absolute",
  "residual_long_conv": true,
  "transformers_

200000ba
Using Monarch Mixer for Sequence Mixing: True
-- Bidirectional: True
-- Using Long Conv Residual: True
-- Hyena w: 10
-- Hyena w mod: 1
-- Hyena filter order: 128
-- Hyena filter dropout: 0.2
-- Hyena filter wd: 0.1
-- Hyena filter emb dim: 5
-- Hyena filter lr: 0.001
-- Hyena filter lr pos emb: 1e-05


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/lukas/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.38.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /home/lukas/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/vocab.tx

In [33]:
from datasets import load_from_disk
from collate import DataCollatorForLanguageModelingSpan
from transformers import AutoTokenizer, Trainer, TrainingArguments

#Load model directly
%env WANDB_PROJECT=singlesamplednam2

tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision="downstream_species_lm")


dataset = load_from_disk("../micro")
dataset = dataset.remove_columns(["species_name", "__index_level_0__"])

data_collator = DataCollatorForLanguageModelingSpan(tokenizer, mlm=True, mlm_probability = 0.02, span_length = 6)

training_args = TrainingArguments(
    output_dir="./results/correct_model_1sample",
    
    max_steps=20000,
    
    seed=17,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    
    logging_strategy="steps",
    logging_steps=1,
    
    remove_unused_columns=False,
    evaluation_strategy="no",
    
    #dataloader_num_workers=0,
    run_name="repo_model_1sample",
    report_to="none" #"wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator
)


trainer.train()

env: WANDB_PROJECT=singlesamplednam2


  0%|          | 0/20000 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

In [31]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset["train"], batch_size=1, shuffle=True, collate_fn=data_collator)
for step, inputs in enumerate(train_dataloader):
    inputs = {
        "args": inputs
    }
    model(**inputs)

TypeError: HuggingFaceModel.forward() got an unexpected keyword argument 'args'